In [1]:
import numpy as np
from icecream import ic
import matplotlib.pyplot as plt
import closest_neighbour
from pyrr import Matrix44
import polyscope as ps

In [113]:
T_true = np.array(Matrix44.look_at([1, 2, 3], [0, 9, 0], [0, 1, 0]).T)
R_true = T_true[:3, :3]
t_true = np.array([-10, 20, 2])

# Generate data as a list of 2d points
num_points = 30
X_true = np.zeros((num_points, 3))
X_true[:, 0] = range(0, num_points)
X_true[:, 1] = 0.2 * X_true[:, 0] * np.sin(0.5 * X_true[:, 0])
X_true[:, 2] = 0.2 * X_true[:, 1] * np.sin(0.5 * X_true[:, 1])
# Move the data
X_moved = X_true @ R_true.T + t_true

# Assign to variables we use in formulas.
Q = X_true
P = X_moved
weight = np.ones(len(P))

In [119]:
# https://web.stanford.edu/class/cs273/refs/umeyama.pdf

Q_mu = np.average(Q, 0, keepdims=True)
P_mu = np.average(P, 0, keepdims=True)

Q_bar = (Q - Q_mu) * weight[..., None]
P_bar = (P - P_mu) * weight[..., None]

assert np.linalg.matrix_rank(Q_bar) >= 3
assert np.linalg.matrix_rank(P_bar) >= 3

_, closest_Q_indices, _, _ = closest_neighbour.compute(P_bar, Q_bar)

N = len(P)
weight_sum = np.sum(weight)

cov = Q_bar[closest_Q_indices].T @ P_bar / N / weight_sum
U, S, V_T = np.linalg.svd(cov)
R = U @ V_T

# reflection
E = np.ones(len(cov))
E[-1] = np.sign(np.linalg.det(R))

R = R * E

s = np.sum(S) / (np.sum(P_bar * P_bar) / N / weight_sum )
t = Q_mu - s * P_mu @ R.T

print(R, s, t)

P = s * P @ R.T + t

# t = Q_mu - P_mu @ R.T
# print(R, t)
# P = P @ R.T + t

residual = ((P - Q)**2).sum(1)
# Huber loss
weight = np.where(residual < 2, np.power(residual, -0.5), 1)

print(np.average(residual))



[[ 9.99968371e-01  7.94866269e-03  2.74118372e-04]
 [-7.94914094e-03  9.99966803e-01  1.79010016e-03]
 [-2.59880369e-04 -1.79222255e-03  9.99998360e-01]] 1.0000932284818638 [[-0.00295877  0.11475733  0.00419144]]
4.0852161308958955e-05


In [121]:
ps.init()
ps.register_point_cloud('Q', Q)
ps.register_point_cloud('P', P)
ps.show()